# Export data to files with Suitcase

In this notebook you will:

* Export saved data to CSV and TIFF.
* See a preview of how exporting data live during a scan works.

## Configuration
Below, we will connect to EPICS IOC(s) controlling simulated hardware in lieu of actual motors, detectors. The IOCs should already be running in the background. Run this command to verify that they are running: it should produce output with RUNNING on each line. In the event of a problem, edit this command to replace `status` with `restart all` and run again.

In [1]:
!supervisorctl -c supervisor/supervisord.conf status

decay                            RUNNING   pid 4914, uptime 0:00:33
mini_beamline                    RUNNING   pid 4915, uptime 0:00:33
random_walk                      RUNNING   pid 4916, uptime 0:00:33
random_walk_horiz                RUNNING   pid 4917, uptime 0:00:33
random_walk_vert                 RUNNING   pid 4930, uptime 0:00:31
simple                           RUNNING   pid 4919, uptime 0:00:33
thermo_sim                       RUNNING   pid 4920, uptime 0:00:33
trigger_with_pc                  RUNNING   pid 4921, uptime 0:00:33


In [2]:
%run scripts/beamline_configuration.py

## Scalar data to CSV

In [3]:
RE(scan([ph], motor_ph, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 3     Time: 2020-05-18 21:07:36
Persistent Unique Scan ID: 'd5f0af41-094a-43e4-81f8-fd13b9bc1cbe'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+------------+
|   seq_num |       time |   motor_ph |     ph_det |
+-----------+------------+------------+------------+
|         1 | 21:07:36.3 |     -1.000 |     101903 |
|         2 | 21:07:36.3 |     -0.500 |     104108 |


|         3 | 21:07:36.4 |      0.000 |     104858 |
|         4 | 21:07:36.4 |      0.500 |     104464 |
|         5 | 21:07:36.4 |      1.000 |     102088 |
+-----------+------------+------------+------------+
generator scan ['d5f0af41'] (scan num: 3)





('d5f0af41-094a-43e4-81f8-fd13b9bc1cbe',)

In [4]:
import suitcase.csv

In [5]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/d5f0af41-094a-43e4-81f8-fd13b9bc1cbe-primary.csv')]}

In [6]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}-{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni-12345-primary.csv')]}

In [7]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary.csv')]}

## Image data to TIFF

In [8]:
RE(scan([spot], mtr_spotx, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 4     Time: 2020-05-18 21:07:36
Persistent Unique Scan ID: '598783b3-20b6-4df8-8f38-31c441a0faa3'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+-------------+------------+
|   seq_num |       time | motor_spotx |   spot_roi |
+-----------+------------+-------------+------------+
|         1 | 21:07:36.8 |          -1 | 1131217588 |


|         2 | 21:07:36.9 |          -0 | 1129344850 |


|         3 | 21:07:37.0 |           0 | 1123919674 |
|         4 | 21:07:37.1 |           0 | 1118591421 |


|         5 | 21:07:37.2 |           1 | 1113579380 |


+-----------+------------+-------------+------------+
generator scan ['598783b3'] (scan num: 4)





('598783b3-20b6-4df8-8f38-31c441a0faa3',)

### TIFF stack, a 3D block of images in one file

In [9]:
import suitcase.tiff_stack

In [10]:
suitcase.tiff_stack.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary-spot_img.tiff')]}

### TIFF series

In [11]:
import suitcase.tiff_series

Unlike CSV or TIFF stack, this exporter creates separate file per row (Event document). We can include information from a specific Event in the filename. Here, we include the motor position.

In [12]:
suitcase.tiff_series.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}/{event[data][motor_spotx]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-1.0-primary-spot_img-0.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-0.5-primary-spot_img-1.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.0-primary-spot_img-2.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.5-primary-spot_img-3.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/1.0-primary-spot_img-4.tiff')]}

## "Beta" feature: Set up live exporting

*We plan to streamline this in the future, but here is a demonstration that it is possible.*

In [13]:
# Execute this once to configure RE to send data to an exporter.
# At the beamline, this would normally be done in a script automatically at startup.

from event_model import RunRouter
from databroker.core import discover_handlers

def factory(name, start_doc):

    serializer = suitcase.tiff_series.Serializer(
        'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-{start[uid]:.8}-')

    def cb(name, doc):
        serializer(name, doc)

    return [cb], []


rr = RunRouter([factory], discover_handlers())
RE.subscribe(rr)

2

In [14]:
RE(count([spot], num=30, delay=1), sample_comp='Lab6', barcode='12345')



Transient Scan ID: 5     Time: 2020-05-18 21:07:37
Persistent Unique Scan ID: '66db511e-fcc4-41eb-acaf-6caed0b75908'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+
|   seq_num |       time |   spot_roi |
+-----------+------------+------------+
|         1 | 21:07:37.7 | 1089970803 |


|         2 | 21:07:38.7 | 1053508375 |


|         3 | 21:07:39.7 | 1097462199 |


...

|        28 | 21:08:04.7 | 1130931373 |


|        29 | 21:08:05.7 | 1083981405 |


|        30 | 21:08:06.7 | 1053370484 |


+-----------+------------+------------+
generator count ['66db511e'] (scan num: 5)





('66db511e-fcc4-41eb-acaf-6caed0b75908',)